<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats2/blob/main/04_diagnosing_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCMCの診断
* サンプリングがうまくいったかどうか、診断する方法がある。
* うまくいっていない場合、パラメータの付け方を変えることで改良できる場合がある。
* MCMCの診断については、下記Webページを参照のこと。
 * https://www.statlect.com/fundamentals-of-statistics/Markov-Chain-Monte-Carlo-diagnostics
 * https://mc-stan.org/docs/reference-manual/analysis.html

## 準備

In [ ]:
!pip install arviz
!pip install git+https://github.com/pyro-ppl/numpyro.git

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import jax
import jax.numpy as jnp
from jax import random
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive

import arviz as az

%config InlineBackend.figure_format = 'retina'

plt.style.use("bmh")
rng_key = random.PRNGKey(0)
az.style.use("arviz-darkgrid")
numpyro.set_platform("cpu")

## サンプリングがうまく行かない理由
* 参考資料
 * https://mc-stan.org/docs/reference-manual/divergent-transitions.html
* HMCは、勾配を使ってfirst-orderの近似を行っているため、high curvatureな場所では、本当に求めたいものから大きく外れてしまうことがある。すると、サンプリングがパラメータ空間内での単なるランダムウォークに近くなってしまい、密度関数の"濃淡"を反映しないものになってしまう。
 * 別の参考資料 https://norimune.net/3149

## 例題1: Neal’s funnel
* 下のような確率分布を考える。 
 * https://mc-stan.org/docs/stan-users-guide/reparameterization.html


$$\begin{align} 
y & \sim N(0, 3^2) \\
x_i & \sim N(0, e^y), \text{ $i=1,\ldots, 9$ }  
\end{align}$$


* 同時分布を式で書くと・・・
$$p(y, x_1, \ldots, x_9) = p(y) \prod_{i=1}^9 p(x_i | y)$$
where
$$ p(y) = \frac{1}{\sqrt{2\pi 3^2}} \exp\bigg( - \frac{y^2}{2 \times 3^2}\bigg)  $$
and
$$ p(x_i | y) = \frac{1}{\sqrt{2\pi e^y}} \exp\bigg( - \frac{x_i^2}{2e^y} \bigg) $$

* この分布の問題点
 * $y$の値が小さいとき、$x_i$の従う正規分布が、非常にpeakyな密度関数を持つ
 * 従って、サンプリングが困難になる。

* 今回は、この分布からサンプリングする（＝この分布に従う乱数を発生させる）。
 * つまり、今日はベイズの話をするのではなく（＝事後分布からのサンプリングをするのではなく）・・・
 * NUTSサンプラーでもうまくいかない場合がどんな場合かを、単に説明する。

### 実装方法 (1)
* これは悪い実装方法。
 * divergent transitionが発生する。
 * さらに、周辺分布$p(y)$は正規分布となるはずなのに、ヒストグラムが全く正規分布の形にならない。

In [ ]:
def model():
  y = numpyro.sample("y", dist.Normal(0, 3))
  x = numpyro.sample("x", dist.Normal(jnp.zeros(9), jnp.exp(y/2)))

* 　今回は、`return_inferencedata=True`と設定せず、arviz向けではない形式、元のPyMC3の形式で、サンプリング結果を得る
 * arviz向けのサンプリング結果で同じようにdivergenceの分析を行う方法は後で説明する。

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_)

In [ ]:
mcmc.print_summary()

* 問題点
 * divergenceの値がゼロになっていない。
 * `y`の`n_eff`が非常に小さい。

In [ ]:
idata = az.from_numpyro(mcmc)
idata

* divergenceとは

> "A divergence arises when the simulated Hamiltonian trajectory departs from the true trajectory as measured by departure of the Hamiltonian value from its initial value. When this divergence is too high, the simulation has gone off the rails and cannot be trusted. The positions along the simulated trajectory after the Hamiltonian diverges will never be selected as the next draw of the MCMC algorithm, potentially reducing Hamiltonian Monte Carlo to a simple random walk and biasing estimates by not being able to thoroughly explore the posterior distribution." ( https://mc-stan.org/docs/reference-manual/divergent-transitions.html )



* PyMC3でのdivergenceのチェック方法

In [ ]:
diverging = idata.sample_stats.diverging.data.flatten()
diverging

In [ ]:
diverging.sum()

In [ ]:
np.where(diverging)

* chainの中でdivergenceが発生したサンプルを赤で示してみる。
 * $x_1$だけに注目してプロットする。

In [ ]:
x1 = idata.posterior['x'].data[:,:,0].flatten()
y = idata.posterior['y'].data.flatten()

plt.figure(figsize=(6, 6))
plt.scatter(x1[~ diverging], y[~ diverging], color='g')
plt.scatter(x1[diverging], y[diverging], color='r')
plt.axis([-20, 20, -9, 9])
plt.ylabel('y')
plt.xlabel('x_1')
plt.title('scatter plot between y and x_1');

* 上のプロットで分かるように、図の下の方の領域で全くサンプルが取られていない。

* $y$のヒストグラムを確認する。
 * yの周辺分布（下の式）は正規分布になるはずだが・・・
$$p(y) = \idotsint p(y, x_1, \ldots, x_9) dx_1 \cdots dx_9$$

In [ ]:
sns.displot(y, kind="kde");

### 実装方法 (2)
* これは、reparameterizationを使うことで改良された実装。
 * divergent transitionは起こらない。
 * $y$のサンプルのヒストグラムも正規分布の形を示す。

* 元のモデルは
$$\begin{align} 
y & \sim N(0, 3^2) \\
x_i & \sim N(0, e^y), \text{ $i=1,\ldots, 9$ }  
\end{align}$$


* $x_i$を$N(0, e^y)$からサンプリングする、という実装をやめて、代わりに
 * まず$x_\text{raw}$を$N(0,1)$からサンプリングし・・・
 * その$x_\text{raw}$を$x = e^{y/2} x_\text{raw}$という式で変換している。

In [ ]:
def model_revised():
  y = numpyro.sample("y", dist.Normal(0, 3))
  x_raw = numpyro.sample("x_raw", dist.Normal(jnp.zeros(9), 1))
  x = numpyro.deterministic("x", jnp.exp(y/2) * x_raw)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model_revised), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_)

In [ ]:
mcmc.print_summary()

In [ ]:
idata = az.from_numpyro(mcmc)

diverging = idata.sample_stats.diverging.data.flatten()
x1 = idata.posterior['x'].data[:,:,0].flatten()
y = idata.posterior['y'].data.flatten()

In [ ]:
diverging.sum()

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x1[~ diverging], y[~ diverging], color='g')
plt.scatter(x1[diverging], y[diverging], color='r')
plt.axis([-20, 20, -9, 9])
plt.ylabel('y')
plt.xlabel('x_1')
plt.title('scatter plot between y and x_1');

In [ ]:
sns.displot(y, kind="kde");

## 例題2: The Eight Schools Model
* 下記Webページにある「The Eight Schools Model」の実験を再現してみる。
 * https://github.com/pymc-devs/pymc-examples/blob/main/examples/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.ipynb




> "Hamiltonian Monte Carlo, for example, is especially powerful in this regard as its failures to be geometrically ergodic with respect to any target distribution manifest in distinct behaviors that have been developed into sensitive diagnostics. One of these behaviors is the appearance of divergences that indicate the Hamiltonian Markov chain has encountered regions of high curvature in the target distribution which it cannot adequately explore."



* この例題は、NumPyroのサイトでも触れられている。
 * https://num.pyro.ai/en/latest/getting_started.html

### データセット
* `y`が観測データを表す確率変数
 * 各校で同じコーチングを実施し、その前後で学力テストの点数がどう変化したかを表す。
 * 正確には、この`y`は観測データではなく、8つの学校ごとに別々の回帰分析によって得られたestimates。
 * cf. https://arxiv.org/abs/1507.04544 のSection 4.1
* `sigma`が既知のパラメータ
 * これも、8つの学校ごとに別々の回帰分析によって得られたstandard errors。

In [ ]:
# Data of the Eight Schools Model
y = jnp.asarray([28,  8, -3,  7, -1,  1, 18, 12], dtype=float)
sigma = jnp.asarray([15, 10, 16, 11,  9, 11, 10, 18], dtype=float)
J = y.shape[0]

* このデータを以下のようにモデリングする。
$$\begin{align}
\mu & \sim N(0,5^2) \notag \\
\tau & \sim \text{Half-Cauchy}(5) \\
\theta_n & \sim N(\mu, \tau^2) \\
y_n & \sim N(\theta_n, \sigma_n^2)
\end{align}$$
 * $y_n$が上記コードの`y`に対応する。
 * $\sigma_n$が上記コードの`sigma`に対応する。

* Half-cauchy分布については下記ページを参照
 * https://distribution-explorer.github.io/continuous/halfcauchy.html
> "The Half-Cauchy distribution with 𝜇=0 is a useful prior for nonnegative parameters that may be very large, as allowed by the very heavy tails of the Half-Cauchy distribution."

### 実装方法(1)
* これは悪い実装方法。
 * 後でreparameterizationを使って改良する。

In [ ]:
def model(y=None):
  mu = numpyro.sample("mu", dist.Normal(0, 5))
  tau = numpyro.sample("tau", dist.HalfCauchy(5))
  with numpyro.plate("J", J):
    theta = numpyro.sample("theta", dist.Normal(mu, tau))
    numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, y=y)

In [ ]:
mcmc.print_summary()

In [ ]:
idata = az.from_numpyro(mcmc)

* `r_hat`が1.0から離れているものもある。
 * https://mc-stan.org/docs/reference-manual/analysis.html
* 特に`tau`に問題がありそう。

In [ ]:
az.plot_trace(idata);

* このデータについては$\tau$の"真の値"が分かっているらしい。
 * "真の値"の詳細は https://discourse.pymc.io/t/how-is-the-true-value-of-tau-in-the-eight-schools-model-known/1932
* logスケールでプロットして"真の値"からのズレを見てみる。
 * $\log \tau$の真の値は0.7657852らしいです。

In [ ]:
logtau = np.log(idata.posterior['tau'].data)
plt.figure(figsize=(10, 3))
plt.axhline(0.7657852, lw=2.5, color='gray')
for j in range(logtau.shape[0]):
  mlogtau = [np.mean(logtau[j,:i]) for i in np.arange(1, len(logtau[j]))]
  plt.plot(mlogtau, lw=2)
plt.ylim(0, 2)
plt.xlabel('Iteration')
plt.ylabel('MCMC mean of log(tau)')
plt.title('MCMC estimation of log(tau)');

* いくつかのchainで、divergenceが発生したサンプルをプロットしてみる。

In [ ]:
chain_id = 0

diverging = idata.sample_stats.diverging.data[chain_id]
theta0 = idata.posterior['theta'][chain_id][:,0]

plt.figure(figsize=(6, 4))
plt.scatter(theta0[~diverging], logtau[chain_id][~diverging], color='g')
plt.scatter(theta0[diverging], logtau[chain_id][diverging], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

In [ ]:
chain_id = 1

diverging = idata.sample_stats.diverging.data[chain_id]
theta0 = idata.posterior['theta'][chain_id][:,0]

plt.figure(figsize=(6, 4))
plt.scatter(theta0[~diverging], logtau[chain_id][~diverging], color='g')
plt.scatter(theta0[diverging], logtau[chain_id][diverging], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

#### 自己相関
 * 自己相関は小さいほど良い。小さいほど、サンプルが相互に独立だとみなせる。
 * 比較的大きなラグ(lag)でも相関が0に近くない場合は、問題あり。
 * 参考資料
  * https://www.statlect.com/fundamentals-of-statistics/autocorrelation

In [ ]:
az.plot_autocorr(idata);

### 実装方法(2)
* reparameterizationを使う。

* 今度は以下のようにモデリングする。
$$\begin{align}
\mu & \sim N(0,5^2) \notag \\
\tau & \sim \text{Half-Cauchy}(5) \\
\tilde{\theta_n} & \sim N(0, 1) \\
\theta_n & = \mu + \tau \cdot \tilde{\theta_n} \\
y_n & \sim N(\theta_n, \sigma_n^2)
\end{align}$$


* https://github.com/pymc-devs/pymc-examples/blob/main/examples/diagnostics_and_criticism/Diagnosing_biased_Inference_with_Divergences.ipynb
> In a non-centered parameterization we do not try to fit the group-level parameters directly, rather we fit a latent Gaussian variable from which we can recover the group-level parameters with a scaling and a translation.


* $\theta_n$を$N(\mu, \tau^2)$からサンプリングする、という実装をやめて、代わりに
 * まず$\tilde{\theta_n}$を$N(0,1)$からサンプリングし・・・
 * その$\tilde{\theta_n}$を$\theta_n = \mu + \tau \tilde{\theta_n}$という式で変換している。

In [ ]:
def model(y=None):
  mu = numpyro.sample("mu", dist.Normal(0, 5))
  tau = numpyro.sample("tau", dist.HalfCauchy(5))
  with numpyro.plate("J", J):
    theta_tilde = numpyro.sample("theta_tilde", dist.Normal(0, 1))
    theta = numpyro.deterministic("theta", mu + tau * theta_tilde)
    numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
mcmc = MCMC(NUTS(model), num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, y=y)

In [ ]:
mcmc.print_summary()

In [ ]:
idata_revised = az.from_numpyro(mcmc)

In [ ]:
az.plot_trace(idata_revised);

In [ ]:
logtau = np.log(idata_revised.posterior['tau'].data)
plt.figure(figsize=(10, 3))
plt.axhline(0.7657852, lw=2.5, color='gray')
for j in range(logtau.shape[0]):
  mlogtau = [np.mean(logtau[j,:i]) for i in np.arange(1, len(logtau[j]))]
  plt.plot(mlogtau, lw=2)
plt.ylim(0, 2)
plt.xlabel('Iteration')
plt.ylabel('MCMC mean of log(tau)')
plt.title('MCMC estimation of log(tau)');

In [ ]:
chain_id = 0

diverging = idata_revised.sample_stats.diverging.data[chain_id]
theta0 = idata_revised.posterior['theta'][chain_id][:,0]

plt.figure(figsize=(6, 4))
plt.scatter(theta0[~diverging], logtau[chain_id][~diverging], color='g')
plt.scatter(theta0[diverging], logtau[chain_id][diverging], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

In [ ]:
chain_id = 1

diverging = idata_revised.sample_stats.diverging.data[chain_id]
theta0 = idata_revised.posterior['theta'][chain_id][:,0]

plt.figure(figsize=(6, 4))
plt.scatter(theta0[~diverging], logtau[chain_id][~diverging], color='g')
plt.scatter(theta0[diverging], logtau[chain_id][diverging], color='r')
plt.axis([-20, 50, -6, 4])
plt.ylabel('log(tau)')
plt.xlabel('theta[0]')
plt.title('scatter plot between log(tau) and theta[0]');

#### 自己相関
* 自己相関も改善されている。

In [ ]:
az.plot_autocorr(idata_revised);

### HDI (highest density interval)
* ArviZのforest plotで、chainごとのHDI (highest density interval) を可視化する。
* HDIが何であるかについては下記を参照。
 * http://web.sfc.keio.ac.jp/~maunz/BS14/BS14-11.pdf
 * https://www.sciencedirect.com/topics/mathematics/highest-density-interval
* 改良後のモデルのほうが、chainごとのHDIのばらつきが少ないように見える。

In [ ]:
az.plot_forest(
    [idata, idata_revised],
    model_names=["centered", "non centered"],
    labeller=az.labels.DimCoordLabeller(),
    figsize=(10,10),
    legend=False,
    );